In [1]:
import torch
import sys
print("GPU Available: ", torch.cuda.is_available())
import contractions
from datasets import load_dataset
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from gensim.models import KeyedVectors
from scipy.sparse import hstack
from nltk import pos_tag
from sklearn.preprocessing import StandardScaler
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

GPU Available:  True


C:\Users\melih\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\melih\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data=pd.read_csv('D:/Python_Projects/spotivibe_exp/data.csv')
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_lyrics'], data['mood_cats'], test_size=0.2, random_state=42)

# Unigram + Bigram TFIDF SVM

In [18]:
# TF-IDF Vectorizer for Unigrams and Bigram
bigram_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_tfidf = bigram_vectorizer.fit_transform(X_train)
X_test_tfidf = bigram_vectorizer.transform(X_test)

svc_model = SVC(kernel='rbf',C=10, gamma=1)
y_pred_bigram = svc_model.predict(X_test_tfidf)
print("Bigram Classification Report:")
print(classification_report(y_test, y_pred_bigram))

In [6]:
# Load pre-trained Word2Vec model (Google News, 300 dimensions)
word2vec_model = KeyedVectors.load_word2vec_format('D:/GoogleNews-vectors-negative300.bin', binary=True)

def get_average_word2vec(text, model, vector_size=300):
    # Tokenize the text and calculate average word2vec for all valid words
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # Return a vector of zeros if no valid words found
        return np.zeros(vector_size)

# Apply the function to the training and test sets
X_train_w2v = np.array([get_average_word2vec(text, word2vec_model) for text in X_train])
X_test_w2v = np.array([get_average_word2vec(text, word2vec_model) for text in X_test])

# Word2Vec SVM

In [10]:
# Load pre-trained Word2Vec model (Google News, 300 dimensions)
word2vec_model = KeyedVectors.load_word2vec_format('D:/GoogleNews-vectors-negative300.bin', binary=True)

def get_average_word2vec(text, model, vector_size=300):
    # Tokenize the text and calculate average word2vec for all valid words
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # Return a vector of zeros if no valid words found
        return np.zeros(vector_size)

# Apply the function to the training and test sets
X_train_w2v = np.array([get_average_word2vec(text, word2vec_model) for text in X_train])
X_test_w2v = np.array([get_average_word2vec(text, word2vec_model) for text in X_test])

# Standardize the Word2Vec embeddings
scaler = StandardScaler()
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_test_w2v_scaled = scaler.transform(X_test_w2v)

svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_w2v_scaled, y_train)

# Evaluate the model
y_pred = svc_model.predict(X_test_w2v_scaled)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.03       842
           1       0.45      1.00      0.62       779
           2       1.00      0.89      0.94       632
           3       1.00      0.95      0.97       676

    accuracy                           0.68      2929
   macro avg       0.86      0.71      0.64      2929
weighted avg       0.85      0.68      0.60      2929



# NRClex Lexicons SVM

In [13]:
from nrclex import NRCLex

def get_emotion_scores(text):
    emotion = NRCLex(text)
    return emotion.raw_emotion_scores

# Apply the function to get emotion scores for each lyric
def extract_emotion_features(texts):
    all_scores = []
    for text in texts:
        scores = get_emotion_scores(text)
        # You can standardize or normalize the scores if needed
        all_scores.append([scores.get('anger', 0), scores.get('fear', 0), scores.get('joy', 0), scores.get('sadness', 0)])
    return np.array(all_scores)


# Get emotion scores for training and test sets
X_train_nrc = extract_emotion_features(X_train)
X_test_nrc = extract_emotion_features(X_test)

scaler = StandardScaler()
X_train_nrc_s = scaler.fit_transform(X_train_nrc)
X_test_nrc_s = scaler.transform(X_test_nrc)

svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_nrc_s, y_train)

# Evaluate the model
y_pred = svc_model.predict(X_test_nrc_s)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.40      0.41       842
           1       0.30      0.67      0.42       779
           2       0.67      0.01      0.02       632
           3       0.48      0.24      0.32       676

    accuracy                           0.35      2929
   macro avg       0.46      0.33      0.29      2929
weighted avg       0.45      0.35      0.31      2929



In [16]:
X_train_nrc_s.shape

(11714, 4)

# Unigram+Bigram TFIDF Combined Word2Vec SVM


In [12]:
# Combine the dense Word2Vec embeddings and sparse TF-IDF features
X_train_combined = hstack([X_train_tfidf, X_train_w2v_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_w2v_scaled])

svc_model = SVC(kernel='rbf', C=50, gamma=1)
svc_model.fit(X_train_combined, y_train)

# Evaluate the model
y_pred = svc_model.predict(X_test_combined)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.45      1.00      0.62       770
           1       0.67      0.00      0.00       819
           2       1.00      0.91      0.95       671
           3       1.00      0.94      0.97       687

    accuracy                           0.69      2947
   macro avg       0.78      0.71      0.64      2947
weighted avg       0.76      0.69      0.61      2947



# LIWC, Unigram+Bigram TFIDF Combined Word2Vec SVM

In [13]:
# Define LIWC-like categories
pronouns = {'i', 'you', 'he', 'she', 'we', 'they'}
function_words = {'the', 'and', 'but', 'or', 'so', 'because', 'when', 'although', 'if'}
positive_emotions = {'happy', 'joy', 'love', 'great', 'wonderful', 'good'}
negative_emotions = {'sad', 'angry', 'fear', 'hate', 'bad', 'terrible', 'worried'}

def extract_liwc_features(text):
    words = text.split()
    num_words = len(words)
    
    # Initialize counters for each category
    pronoun_count = 0
    function_word_count = 0
    positive_emotion_count = 0
    negative_emotion_count = 0
    
    # Loop through each word in the text
    for word in words:
        word = word.lower()  # Normalize to lowercase
        if word in pronouns:
            pronoun_count += 1
        if word in function_words:
            function_word_count += 1
        if word in positive_emotions:
            positive_emotion_count += 1
        if word in negative_emotions:
            negative_emotion_count += 1
    
    # Calculate percentages relative to total word count
    pronoun_percentage = pronoun_count / num_words if num_words > 0 else 0
    function_word_percentage = function_word_count / num_words if num_words > 0 else 0
    positive_emotion_percentage = positive_emotion_count / num_words if num_words > 0 else 0
    negative_emotion_percentage = negative_emotion_count / num_words if num_words > 0 else 0
    
    # Return a feature vector
    return np.array([num_words, pronoun_percentage, function_word_percentage, 
                     positive_emotion_percentage, negative_emotion_percentage])

# Apply the function to the dataset
X_train_liwc = np.array([extract_liwc_features(text) for text in X_train])
X_test_liwc = np.array([extract_liwc_features(text) for text in X_test])

# Combine TF-IDF features, Word2Vec features, and LIWC-like features
X_train_combined = hstack([X_train_tfidf, np.hstack((X_train_w2v, X_train_liwc))])
X_test_combined = hstack([X_test_tfidf, np.hstack((X_test_w2v, X_test_liwc))])

# Train and evaluate the model
svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_combined, y_train)
y_pred_combined = svc_model.predict(X_test_combined)

# Evaluate the model
print("Classification Report with LIWC-like Features:")
print(classification_report(y_test, y_pred_combined))

Classification Report with LIWC-like Features:
              precision    recall  f1-score   support

           0       0.53      0.58      0.55       770
           1       0.55      0.57      0.56       819
           2       0.98      0.91      0.94       671
           3       0.99      0.94      0.96       687

    accuracy                           0.73      2947
   macro avg       0.76      0.75      0.76      2947
weighted avg       0.75      0.73      0.74      2947



In [19]:
# Define expanded LIWC-like categories
pronouns = {'i', 'you', 'he', 'she', 'we', 'they', 'me', 'him', 'her', 'us', 'them'}
function_words = {'the', 'and', 'but', 'or', 'so', 'because', 'when', 'although', 'if', 'a', 'an'}
positive_emotions = {'happy', 'joy', 'love', 'great', 'wonderful', 'good', 'excited', 'delight', 'glad'}
negative_emotions = {'sad', 'angry', 'fear', 'hate', 'bad', 'terrible', 'worried', 'anxious', 'upset'}
cognitive_processes = {'think', 'know', 'believe', 'understand', 'realize', 'consider', 'decide', 'conclude'}
social_words = {'friend', 'family', 'talk', 'meet', 'chat', 'parent', 'child', 'relationship', 'people'}
temporal_words = {'today', 'now', 'soon', 'tomorrow', 'yesterday', 'later', 'always', 'never', 'before', 'after'}
certainty = {'always', 'never', 'definitely', 'certain', 'sure'}
negation = {'no', 'not', 'never', 'none', 'nobody'}

# Function to extract LIWC-like features
def extract_liwc_features(text):
    words = text.split()
    num_words = len(words)
    
    # Initialize counters for each category
    pronoun_count = 0
    function_word_count = 0
    positive_emotion_count = 0
    negative_emotion_count = 0
    cognitive_process_count = 0
    social_word_count = 0
    temporal_word_count = 0
    certainty_count = 0
    negation_count = 0
    
    # Loop through each word in the text
    for word in words:
        word = word.lower()  # Normalize to lowercase
        if word in pronouns:
            pronoun_count += 1
        if word in function_words:
            function_word_count += 1
        if word in positive_emotions:
            positive_emotion_count += 1
        if word in negative_emotions:
            negative_emotion_count += 1
        if word in cognitive_processes:
            cognitive_process_count += 1
        if word in social_words:
            social_word_count += 1
        if word in temporal_words:
            temporal_word_count += 1
        if word in certainty:
            certainty_count += 1
        if word in negation:
            negation_count += 1
    
    # Calculate percentages relative to total word count
    pronoun_percentage = pronoun_count / num_words if num_words > 0 else 0
    function_word_percentage = function_word_count / num_words if num_words > 0 else 0
    positive_emotion_percentage = positive_emotion_count / num_words if num_words > 0 else 0
    negative_emotion_percentage = negative_emotion_count / num_words if num_words > 0 else 0
    cognitive_process_percentage = cognitive_process_count / num_words if num_words > 0 else 0
    social_word_percentage = social_word_count / num_words if num_words > 0 else 0
    temporal_word_percentage = temporal_word_count / num_words if num_words > 0 else 0
    certainty_percentage = certainty_count / num_words if num_words > 0 else 0
    negation_percentage = negation_count / num_words if num_words > 0 else 0
    
    # Return a feature vector
    return np.array([num_words, pronoun_percentage, function_word_percentage, 
                     positive_emotion_percentage, negative_emotion_percentage,
                     cognitive_process_percentage, social_word_percentage, 
                     temporal_word_percentage, certainty_percentage, negation_percentage])

# Apply the function to the dataset
X_train_liwc = np.array([extract_liwc_features(text) for text in X_train])
X_test_liwc = np.array([extract_liwc_features(text) for text in X_test])

# Combine TF-IDF features, Word2Vec features, and LIWC-like features
X_train_combined = hstack([X_train_tfidf, np.hstack((X_train_w2v, X_train_liwc))])
X_test_combined = hstack([X_test_tfidf, np.hstack((X_test_w2v, X_test_liwc))])

# Train and evaluate the model
svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_combined, y_train)
y_pred_combined = svc_model.predict(X_test_combined)

# Evaluate the model
print("Classification Report with LIWC-like Features:")
print(classification_report(y_test, y_pred_combined))

Classification Report with LIWC-like Features:
              precision    recall  f1-score   support

           0       0.57      0.56      0.56       842
           1       0.52      0.59      0.55       779
           2       0.99      0.89      0.94       632
           3       1.00      0.95      0.97       676

    accuracy                           0.73      2929
   macro avg       0.77      0.75      0.76      2929
weighted avg       0.75      0.73      0.74      2929



# Weighting outperformed classes 0 and 1

In [15]:
# Use TF-IDF Vectorizer to get unigram and bigram features
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Get the list of features (words or n-grams)
features = vectorizer.get_feature_names_out()

# Extract features specific to class 0 (happy) and class 1 (sad)
happy_indices = y_train == 0
sad_indices = y_train == 1

# Calculate the average TF-IDF scores for happy and sad songs
happy_tfidf_avg = X_train_tfidf[happy_indices].mean(axis=0)
sad_tfidf_avg = X_train_tfidf[sad_indices].mean(axis=0)

# Identify unique features for happy and sad songs
happy_unique_indices = np.where(happy_tfidf_avg > sad_tfidf_avg)[1]
sad_unique_indices = np.where(sad_tfidf_avg > happy_tfidf_avg)[1]

# Assign higher weights to these unique features
X_train_weighted = X_train_tfidf.copy()
X_train_weighted[:, happy_unique_indices] *= 1.5  
X_train_weighted[:, sad_unique_indices] *= 1.5  
##When try to increase weight more than 1.5 performance drops.
# Train the SVM model with weighted features
svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_weighted, y_train)

# Evaluate the model
X_test_weighted = X_test_tfidf.copy()
X_test_weighted[:, happy_unique_indices] *= 1.5
X_test_weighted[:, sad_unique_indices] *= 1.5

y_pred_weighted = svc_model.predict(X_test_weighted)
print(classification_report(y_test, y_pred_weighted))

              precision    recall  f1-score   support

           0       0.45      0.95      0.61       770
           1       0.40      0.03      0.06       819
           2       0.99      0.91      0.95       671
           3       1.00      0.94      0.96       687

    accuracy                           0.68      2947
   macro avg       0.71      0.71      0.65      2947
weighted avg       0.69      0.68      0.62      2947



# GloVe SVM

In [3]:
# Path to your GloVe file (300d vectors)
glove_file = 'D:/glove.6B.300d.txt'

# Initialize an empty dictionary to store embeddings
glove_embeddings = {}

# Load GloVe embeddings
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

print(f"Loaded {len(glove_embeddings)} word vectors.")

Loaded 400001 word vectors.


In [4]:
def get_average_glove(text, embeddings, vector_size=300):
    words = text.split()
    word_vectors = [embeddings[word] for word in words if word in embeddings]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # Return a vector of zeros if no valid words found
        return np.zeros(vector_size)

X_train_glove = np.array([get_average_glove(text, glove_embeddings) for text in X_train])
X_test_glove = np.array([get_average_glove(text, glove_embeddings) for text in X_test])

scaler = StandardScaler()
X_train_glove_scaled = scaler.fit_transform(X_train_glove)
X_test_glove_scaled = scaler.transform(X_test_glove)

svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_glove_scaled, y_train)

y_pred = svc_model.predict(X_test_glove_scaled)
print("Classification Report with GloVe embeddings:")
print(classification_report(y_test, y_pred))

Classification Report with GloVe embeddings:
              precision    recall  f1-score   support

           0       1.00      0.01      0.03       842
           1       0.45      1.00      0.62       779
           2       1.00      0.89      0.94       632
           3       1.00      0.95      0.97       676

    accuracy                           0.68      2929
   macro avg       0.86      0.71      0.64      2929
weighted avg       0.85      0.68      0.60      2929



# GloVe Combined Word2Vec SVM

In [7]:
# Step 2: Concatenate GloVe and Word2Vec embeddings
X_train_combined = np.hstack([X_train_glove, X_train_w2v])
X_test_combined = np.hstack([X_test_glove, X_test_w2v])

# Step 3: Scale the combined embeddings
scaler = StandardScaler()
X_train_combined_scaled = scaler.fit_transform(X_train_combined)
X_test_combined_scaled = scaler.transform(X_test_combined)

# Step 4: Train the SVM model on the combined embeddings
svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_combined_scaled, y_train)

# Step 5: Make predictions and evaluate the model
y_pred_combined = svc_model.predict(X_test_combined_scaled)
print("Classification Report with Combined GloVe and Word2Vec embeddings:")
print(classification_report(y_test, y_pred_combined))

Classification Report with Combined GloVe and Word2Vec embeddings:
              precision    recall  f1-score   support

           0       1.00      0.01      0.03       842
           1       0.45      1.00      0.62       779
           2       1.00      0.89      0.94       632
           3       1.00      0.95      0.97       676

    accuracy                           0.68      2929
   macro avg       0.86      0.71      0.64      2929
weighted avg       0.85      0.68      0.60      2929



In [32]:
X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_glove, X_train_w2v])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_glove, X_test_w2v])

scaler = StandardScaler()
X_train_combined_scaled = scaler.fit_transform(X_train_combined)
X_test_combined_scaled = scaler.transform(X_test_combined)

svc_model = SVC(kernel='rbf', C=10, gamma=1)
svc_model.fit(X_train_combined_scaled, y_train)

y_pred_combined = svc_model.predict(X_test_combined_scaled)
print("Classification Report with Combined TF-IDF, GloVe, and Word2Vec embeddings:")
print(classification_report(y_test, y_pred_combined))

Classification Report with Combined TF-IDF, GloVe, and Word2Vec embeddings:
              precision    recall  f1-score   support

           0       1.00      0.01      0.03       842
           1       0.45      1.00      0.62       779
           2       1.00      0.89      0.94       632
           3       1.00      0.95      0.97       676

    accuracy                           0.68      2929
   macro avg       0.86      0.71      0.64      2929
weighted avg       0.85      0.68      0.60      2929

